<a href="https://colab.research.google.com/github/prometricas/Abeba_Datos_Masivos/blob/main/puntosEquipo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Instalación de Java y Hadoop**

## **Instalar JDK de Java**

In [ ]:
!apt-get install -y openjdk-11-jdk-headless -qq > /dev/null

## **Instalar Hadoop**

In [ ]:
# Descargar Hadoop
!wget https://downloads.apache.org/hadoop/common/hadoop-3.4.2/hadoop-3.4.2.tar.gz

--2026-01-11 14:43:55--  https://downloads.apache.org/hadoop/common/hadoop-3.4.2/hadoop-3.4.2.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.208.237, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1065831750 (1016M) [application/x-gzip]
Saving to: ‘hadoop-3.4.2.tar.gz’

hadoop-3.4.2.tar.gz 100%[===================>]   1016M  25.3MB/s    in 41s     

2026-01-11 14:44:37 (24.8 MB/s) - ‘hadoop-3.4.2.tar.gz’ saved [1065831750/1065831750]



In [ ]:
# Descomprimir y mover a carpeta de usuario
!tar -xzf hadoop-3.4.2.tar.gz
!mv hadoop-3.4.2/ /usr/local/

# Mostrar los archivos de carpeta local:
!ls /usr/local

bin    dist_metrics.pxd  games	       include	libexec     man  sbin	src
colab  etc		 hadoop-3.4.2  lib	LICENSE.md  opt  share


## **Definir variables de entorno**


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.4.2"
os.environ["PATH"] += os.pathsep + "/usr/local/hadoop-3.4.2/bin"
!hadoop version

Hadoop 3.4.2
Source code repository https://github.com/apache/hadoop.git -r 84e8b89ee2ebe6923691205b9e171badde7a495c
Compiled by ahmarsu on 2025-08-20T10:30Z
Compiled on platform linux-x86_64
Compiled with protoc 3.23.4
From source with checksum fa94c67d4b4be021b9e9515c9b0f7b6
This command was run using /usr/local/hadoop-3.4.2/share/hadoop/common/hadoop-common-3.4.2.jar


# **2. `Importar base de datos`**

In [ ]:
# Importar archivo
from google.colab import files
files.upload()

# Verificar
!ls -lah
!head -n 5 resultados_futbol.csv

Saving resultados_futbol.csv to resultados_futbol.csv
total 1018M
drwxr-xr-x 1 root root  4.0K Jan 11 14:58 .
drwxr-xr-x 1 root root  4.0K Jan 11 14:33 ..
drwxr-xr-x 4 root root  4.0K Dec 11 14:34 .config
-rw-r--r-- 1 root root 1017M Oct  4 09:30 hadoop-3.4.2.tar.gz
drwxr-xr-x 2 root root  4.0K Jan 11 14:57 .ipynb_checkpoints
-rw-r--r-- 1 root root  557K Jan 11 14:58 resultados_futbol.csv
drwxr-xr-x 1 root root  4.0K Dec 11 14:34 sample_data
,Season,Game,Score,Teams,Home Team,Away Team
0,2000-2001,1,1-2,Mallorca - Real Madrid,Mallorca,Real Madrid
1,2000-2001,1,1-2,Valencia - Racing,Valencia,Racing
2,2000-2001,1,1-0,Athletic - Real Betis,Athletic,Real Betis
3,2000-2001,1,0-2,Atlético - Rayo Vallecano,Atlético,Rayo Vallecano


# **3. Crear scripts: *Mapper, Combiner, Reducer***


## **Mapper**
* Lee cada partido del CSV, calcula los puntos de local y visitante, y emite pares <equipo, puntos>.
* Omite la cabecera y cualquier línea incompleta para evitar ruido en el cálculo.


In [ ]:
%%writefile mapperPuntosEquipo.py
#!/usr/bin/python3
import sys
import csv
import re

pat_equipo = re.compile(r"\s*[-–]\s*")  # Soportar '-' y '–' cuando toca separar equipos.

for linea in sys.stdin:
    linea = linea.strip()
    if not linea:
        continue

    fila = next(csv.reader([linea]))

    # Saltar la cabecera típica del CSV
    if len(fila) >= 2 and fila[1].strip().lower() == "season":
        continue

    try:
        # Priorizo Home Team / Away Team si vienen en el CSV (en este dataset vienen).
        if len(fila) >= 7:
            score = fila[3].strip()
            local = fila[5].strip()
            visita = fila[6].strip()
        elif len(fila) >= 4:
            score = fila[2].strip()
            equipos = fila[3].strip()
            partes = pat_equipo.split(equipos, maxsplit=1)
            if len(partes) != 2:
                continue
            local, visita = partes[0].strip(), partes[1].strip()
        else:
            continue

        g_local, g_visita = [int(x.strip()) for x in score.split("-", 1)]

        if g_local > g_visita:
            p_local, p_visita = 3, 0
        elif g_local < g_visita:
            p_local, p_visita = 0, 3
        else:
            p_local, p_visita = 1, 1

        print(f"{local}\t{p_local}")
        print(f"{visita}\t{p_visita}")

    except Exception:
        # Ignoro líneas con formato inesperado para no tumbar el job.
        continue


Writing mapperPuntosEquipo.py


## **Combiner**
* Recibe par **<equipo, puntos>** ya ordenados por equipo, y acumula puntos por equipo.
* Emite **<equipo, suma_parcial>** para optimizar lo que viaja al Reducer.


In [ ]:
%%writefile combinerPuntosEquipo.py
#!/usr/bin/python3
import sys

equipo_actual = None
suma_actual = 0

for linea in sys.stdin:
    linea = linea.strip()
    if not linea:
        continue

    try:
        equipo, pts = linea.split("\t", 1)
        pts = int(pts)
    except Exception:
        continue

    if equipo_actual is None:
        equipo_actual = equipo
        suma_actual = pts
        continue

    if equipo == equipo_actual:
        suma_actual += pts
    else:
        print(f"{equipo_actual}\t{suma_actual}")
        equipo_actual = equipo
        suma_actual = pts

if equipo_actual is not None:
    print(f"{equipo_actual}\t{suma_actual}")


Writing combinerPuntosEquipo.py


## **Reducer**
* Recibe **<equipo, suma_parcial>** y hace la suma final por equipo.
* Emite el formato final solicitado: **equipo;puntos**.


In [ ]:
%%writefile reducerPuntosEquipo.py
#!/usr/bin/python3
import sys

equipo_actual = None
suma_actual = 0

for linea in sys.stdin:
    linea = linea.strip()
    if not linea:
        continue

    try:
        equipo, pts = linea.split("\t", 1)
        pts = int(pts)
    except Exception:
        continue

    if equipo_actual is None:
        equipo_actual = equipo
        suma_actual = pts
        continue

    if equipo == equipo_actual:
        suma_actual += pts
    else:
        print(f"{equipo_actual};{suma_actual}")
        equipo_actual = equipo
        suma_actual = pts

if equipo_actual is not None:
    print(f"{equipo_actual};{suma_actual}")


Writing reducerPuntosEquipo.py


## Permisos de ejecución
Dar permisos al usuario de consola para ejecutar cada archivo.

In [ ]:
!chmod u+x mapperPuntosEquipo.py
!chmod u+x combinerPuntosEquipo.py
!chmod u+x reducerPuntosEquipo.py

# **4. Hadoop Streaming**
Se fuerza 1 reducer para que la salida mantenga el orden global por clave (alfabético).

In [ ]:
!rm -rf salidaPuntosEquipo

!hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-3.4.2.jar \
  -D mapreduce.job.reduces=1 \
  -file ./mapperPuntosEquipo.py -mapper ./mapperPuntosEquipo.py \
  -file ./combinerPuntosEquipo.py -combiner ./combinerPuntosEquipo.py \
  -file ./reducerPuntosEquipo.py -reducer ./reducerPuntosEquipo.py \
  -input resultados_futbol.csv -output ./salidaPuntosEquipo


2026-01-11 15:16:35,946 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [./mapperPuntosEquipo.py, ./combinerPuntosEquipo.py, ./reducerPuntosEquipo.py] [] /tmp/streamjob746556288062757493.jar tmpDir=null
2026-01-11 15:16:37,657 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2026-01-11 15:16:38,043 INFO mapred.FileInputFormat: Total input files to process : 1
2026-01-11 15:16:38,074 INFO mapreduce.JobSubmitter: number of splits:1
2026-01-11 15:16:38,633 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1823042517_0001
2026-01-11 15:16:38,635 INFO mapreduce.JobSubmitter: Executing with tokens: []
2026-01-11 15:16:39,315 INFO mapred.LocalDistributedCacheManager: Localized file:/content/mapperPuntosEquipo.py as file:/tmp/hadoop-root/mapred/local/job_local1823042517_0001_a8480874-1dea-4079-91c5-1c3f786406cf/mapperPuntosEquipo.py
2026-01-11 15:16:39,380 INFO mapred.LocalDistributedCacheM

In [ ]:
!cat ./salidaPuntosEquipo/part-*

Alavés;467	
Albacete;75	
Almería;283	
Athletic;1242	
Atlético;1463	
Barcelona;1965	
Celta;886	
Cádiz;161	
Córdoba;20	
Deportivo;883	
Deportivo Alavés;31	
Eibar;302	
Elche;184	
Espanyol;1055	
Getafe;840	
Gimnàstic Tarragona;28	
Girona;137	
Granada;359	
Huesca;67	
Hércules;35	
Las Palmas;191	
Leganés;159	
Levante;583	
Mallorca;825	
Málaga;791	
Numancia;148	
Osasuna;865	
R. Sociedad;973	
Racing;524	
Rayo Vallecano;525	
Real Betis;989	
Real Madrid;1959	
Real Murcia;56	
Real Oviedo;86	
Real Sociedad;133	
Real Sporting;237	
Real Valladolid;587	
Real Zaragoza;551	
Recreativo;167	
Sevilla;1386	
Tenerife;74	
Valencia;1449	
Villarreal;1280	
Xerez;34	
